In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')

import datetime


In [4]:
# Trim whitespace from ends of each value across all series in dataframe

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

#df_cust_cluster = trim_all_columns(df_cust_cluster)

In [5]:

df_cust=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_customer.csv',sep=';') 
df_accounts=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_accounts.csv',sep=';')

df_checking_acct=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_checking_acct.csv',sep=';')
df_saving_acct=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_savings_acct.csv',sep=';')
df_credit_acct=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_credit_acct.csv',sep=';')

df_transactions=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_transactions.csv',sep=';')
df_checking_tran=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_checking_tran.csv',sep=';')
df_saving_tran=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_savings_tran.csv',sep=';')
df_credit_tran=pd.read_csv(r'D:\0-LHL-Activities\W05\Mini-project-III\Data\twm_credit_tran.csv',sep=';')

df_checking_acct =trim_all_columns( df_checking_acct ) 
df_saving_acct   =trim_all_columns( df_saving_acct )
df_credit_acct   =trim_all_columns( df_credit_acct )

df_transactions  =trim_all_columns( df_transactions )
df_checking_tran =trim_all_columns( df_checking_tran )
df_saving_tran   =trim_all_columns( df_saving_tran )
df_credit_tran   =trim_all_columns( df_credit_tran )

df_transactions.tran_date=pd.to_datetime(df_transactions.tran_date)
df_checking_tran.tran_date=pd.to_datetime(df_checking_tran.tran_date)
df_saving_tran.tran_date=pd.to_datetime(df_saving_tran.tran_date)
df_credit_tran.tran_date=pd.to_datetime(df_credit_tran.tran_date)



#df_accounts.head(20)b
#df_accounts.info()
#df_accounts['cust_id'].value_counts()
#df_cust['cust_id'].value_counts()
#df_cust.shape
#df_accounts.shape
df_checking_tran.head()

,cust_id,tran_id,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,1363481,26,-0.15,-0.15,0.0,58.99,1995-01-29,,,FK
1,1362784,90,-200.00,-200.00,0.0,1380.69,1995-02-07,53313,A,WD
2,1363088,46,0.00,0.00,0.0,128.35,1995-03-26,834,A,IQ
3,1363306,32,-40.32,-40.32,0.0,433.06,1995-01-29,,P,WD
4,1363251,11,-87.88,-87.88,0.0,1919.55,1995-08-02,,P,WD


In [6]:
df_checking_tran['acct_type'] = 'CK'
df_saving_tran['acct_type']   = 'SV'
df_credit_tran['acct_type']   = 'CC'

df_trans_all=df_checking_tran.append([df_saving_tran,df_credit_tran])
df_trans_all.head()
#df_trans_all.info

df_trans_all_1 = df_trans_all.copy()
df_trans_all_1 = df_trans_all_1.drop('acct_type', axis=1)
df_trans_all_1 = df_trans_all_1.drop('cust_id', axis=1)

df_trans_all_2 = df_transactions.copy()
df_trans_all_2 = df_trans_all_2.drop('acct_nbr', axis=1)

#df_trans_all_1.compare(df_trans_all_2)

df_trans_all_2.head()
df_trans_all_2.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77580 entries, 0 to 77579
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   tran_id        77580 non-null  int64         
 1   tran_amt       77580 non-null  float64       
 2   principal_amt  77580 non-null  float64       
 3   interest_amt   77580 non-null  float64       
 4   new_balance    77580 non-null  float64       
 5   tran_date      77580 non-null  datetime64[ns]
 6   tran_time      77580 non-null  int64         
 7   channel        77580 non-null  object        
 8   tran_code      77580 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 5.3+ MB


In [7]:
df_checking_acct.head()


#df_checking_tran.head()

#a=df_checking_tran.groupby('cust_id').agg({'tran_date':np.max})
#print( datetime.datetime(1996, 1, 1)-a)






,cust_id,acct_nbr,minimum_balance,per_check_fee,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,1362548,13625482,3000,0.00,Y,11.11.1986,NaN,6004.34,569.65
1,1362487,13624872,3000,0.00,Y,12.1.1995,NaN,2781.07,1401.21
2,1363017,13630172,3000,0.00,Y,19.3.1994,NaN,2694.91,147.15
3,1362752,13627522,100,0.15,Y,8.1.1994,NaN,487.69,3.76
4,1363282,13632822,100,0.15,Y,5.3.1988,NaN,133.90,84.18


In [8]:
# Add different info from other tables to df_accounts_1

df_accounts_1=df_accounts.copy()

df_accounts_1['years_with_bank'] = 0
df_accounts_1['num_tran'] = 0
df_accounts_1['dp_tran'] = 0
df_accounts_1['wd_tran'] = 0
df_accounts_1['recency'] = ''



for i in range(len(df_accounts_1.index)):
    Lookfor_cust_id   = df_accounts_1.cust_id[i]
    Lookfor_acct_type = df_accounts_1.acct_type[i]
    #Lookfor_acc_nbr = df_accounts_1.acct_nbr[i]
    
    # Add years_with_bank from df_cust to df_accounts_1
    df_accounts_1.years_with_bank[i] = df_cust[ df_cust.cust_id == Lookfor_cust_id ].years_with_bank
    
    # CK: Add Number of transactions,summation of Positive transactions, summation of Negative transactions
    if Lookfor_acct_type.strip() == 'CK':
        df_accounts_1.num_tran[i] = df_checking_tran.cust_id.loc[(df_checking_tran.cust_id == Lookfor_cust_id)].count()  
        df_accounts_1.dp_tran[i]  = df_checking_tran.tran_amt.loc[(df_checking_tran.cust_id == Lookfor_cust_id) & 
                                                                  (df_checking_tran.tran_amt > 0)].sum()  
        df_accounts_1.wd_tran[i]  = df_checking_tran.tran_amt.loc[(df_checking_tran.cust_id == Lookfor_cust_id) & 
                                                                  (df_checking_tran.tran_amt < 0)].sum()  
        df_accounts_1.recency[i]  = (datetime.datetime(1996, 1, 1)-
                                    (df_checking_tran.tran_date.loc[(df_checking_tran.cust_id == Lookfor_cust_id)].max()))
                                    #/pd.timedelta(1,'D')

                                        
#                                        ('cust_id').agg({'tran_date':np.max})))/np.timedelta64(1,'D')
        

    # SV: Add Number of transactions,summation of Positive transactions, summation of Negative transactions    
    if Lookfor_acct_type.strip() == 'SV':
        df_accounts_1.num_tran[i] = df_saving_tran.cust_id.loc[(df_saving_tran.cust_id == Lookfor_cust_id)].count() 
        df_accounts_1.dp_tran[i]  = df_saving_tran.tran_amt.loc[(df_saving_tran.cust_id == Lookfor_cust_id) & 
                                                                (df_saving_tran.tran_amt > 0)].sum()  
        df_accounts_1.wd_tran[i]  = df_saving_tran.tran_amt.loc[(df_saving_tran.cust_id == Lookfor_cust_id) & 
                                                                (df_saving_tran.tran_amt < 0)].sum()  
        df_accounts_1.recency[i]  = (datetime.datetime(1996, 1, 1)-
                                    (df_saving_tran.tran_date.loc[(df_saving_tran.cust_id == Lookfor_cust_id)].max()))
                                    #/pd.timedelta(1,'D')

    
    # CC: Add Number of transactions,summation of Positive transactions, summation of Negative transactions    
    if Lookfor_acct_type.strip() == 'CC':
        df_accounts_1.num_tran[i] = df_credit_tran.cust_id.loc[(df_credit_tran.cust_id == Lookfor_cust_id)].count() 
        df_accounts_1.dp_tran[i]  = df_credit_tran.tran_amt.loc[(df_credit_tran.cust_id == Lookfor_cust_id) & 
                                                                (df_credit_tran.tran_amt > 0)].sum()                                                                 
        df_accounts_1.wd_tran[i]  = df_credit_tran.tran_amt.loc[(df_credit_tran.cust_id == Lookfor_cust_id) & 
                                                                (df_credit_tran.tran_amt < 0)].sum()  
        df_accounts_1.recency[i]  = (datetime.datetime(1996, 1, 1)-
                                    (df_credit_tran.tran_date.loc[(df_credit_tran.cust_id == Lookfor_cust_id)].max()))
                                    #/pd.timedelta(1,'D')    
df_accounts_1.head()   

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance,years_with_bank,num_tran,dp_tran,wd_tran,recency
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58,3,40,1611.39,-2757.03,1 days 00:00:00
1,4561143213627090,1362709,CC,Y,15.3.1993,NaN,266.34,496.15,4,55,4280.64,-4506.70,10 days 00:00:00
2,4561143213628360,1362836,CC,Y,18.3.1992,NaN,55.90,1000.00,6,41,1122.86,-1937.27,8 days 00:00:00
3,13633112,1363311,CK,Y,6.7.1995,NaN,11017.13,968.46,0,99,5978.89,-16027.56,23 days 00:00:00
4,4561143213633610,1363361,CC,Y,17.6.1994,NaN,849.37,462.28,4,43,5201.82,-4802.77,1 days 00:00:00


In [9]:

#a=df_trans_all.groupby(['acct_type','cust_id']).agg({'cust_id':len})
#print(a)

In [10]:
duplicate = df_transactions[df_transactions.duplicated()]
duplicate

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
